<a href="https://colab.research.google.com/github/iggyloh/Flight-Delays/blob/main/DAC_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Uncomment once and run the file to get access to all the necessary libraries
%pip install -r requirements.txt
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr
!apt-get install -y libmagic1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of unstructured-client to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (196 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123633 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

In [2]:
import os
import bs4
import ast
import uuid
import json
import base64
import markdown
from PIL import Image
from pprint import pprint
from PyPDF2 import PdfMerger
from base64 import b64decode
from datetime import datetime
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from pdf2image import convert_from_path
from IPython.display import Image, display
from unstructured.partition.pdf import partition_pdf
from unstructured.documents.elements import Text, Image
from typing_extensions import Annotated, TypedDict, Sequence

from langchain import hub
from langchain_core.prompts import MessagesPlaceholder
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.text_splitter  import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

from langchain_community.document_loaders import(
  PyPDFLoader,
  Docx2txtLoader,
  UnstructuredPDFLoader,
  WebBaseLoader,
  UnstructuredMarkdownLoader,
  UnstructuredWordDocumentLoader,
  TextLoader,
  UnstructuredPDFLoader
)

from langchain.schema import BaseMessage
from langchain_community.vectorstores import Chroma, InMemoryVectorStore
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_history_aware_retriever
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

from IPython.display import Image, display
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

import gradio as gr

In [3]:


# Tracing via Langsmith
trace = os.getenv("LANGCHAIN_TRACING_V2")
langsmith = os.getenv("LANGCHAIN_API_KEY")

load_dotenv('template.env')

# Build a GPT model
gpt = ChatOpenAI(
    model = 'gpt-4o',
    temperature = 0.1)

# Embedding model
embeddings = OpenAIEmbeddings(
    model = 'text-embedding-3-large',
    openai_api_keys = os.getenv('OPENAI_API_KEY')
)

# Small test
response = gpt.invoke('Who won the most recent world cup')
print(response.content)

<ipython-input-3-089222b8c077>:13: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(
/usr/local/lib/python3.10/dist-packages/langchain_community/embeddings/openai.py:271: UserWarning: WARNING! openai_api_keys is not default parameter.
                    openai_api_keys was transferred to model_kwargs.
                    Please confirm that openai_api_keys is what you intended.
  warnings.warn(


As of my last update, the most recent FIFA World Cup took place in 2022 in Qatar, and Argentina won the tournament by defeating France in the final. If you are referring to another sport or a different event, please provide more details.


In [6]:
import pandas as pd
# Path to the uploaded file
file_path = "/content/updated_papers.csv"

# Read the CSV into a Pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows
print(df.head())




  source_id  year                                              title  \
0        27  1987                         Bit-Serial Neural Networks   
1        63  1987                        Connectivity Versus Entropy   
2        60  1987        The Hopfield Model with Multi-Level Neurons   
3        59  1987                               How Neural Nets Work   
4        69  1987  Spatial Organization of Neural Networks: A Pro...   

                                            abstract  \
0  A  bit  - serial  VLSI  neural  network  is  d...   
1  How  does  the  connectivity  of a  neural  ne...   
2  The  Hopfield  neural  network.  model  for  a...   
3  There is  presently great interest in the abil...   
4  The  aim  of  this  paper  is  to  explore  th...   

                                           full_text encoding  \
0  573 \n\nBIT - SERIAL NEURAL  NETWORKS \n\nAlan...    utf-8   
1  1 \n\nCONNECTIVITY VERSUS ENTROPY \n\nYaser  S...    utf-8   
2  278 \n\nTHE HOPFIELD MODEL WITH 

<ipython-input-6-7fd97b36c5b9>:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [4]:
!pip install chromadb

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,       # Maximum number of characters per chunk
    chunk_overlap=200,    # Number of overlapping characters between chunks
    separators=["\n\n", "\n", " ", ""]  # Order of separators to try splitting
)

In [21]:
test_df = df.loc[df['year'] == '2000']


In [22]:
chunks_list = []  # Store chunks with metadata

for index, row in test_df.iterrows():  # Iterate over the filtered DataFrame
    if pd.notna(row['full_text']):  # Ensure the text is not NaN
        # Split the text into chunks
        chunks = text_splitter.split_text(row['full_text'])
        for chunk in chunks:
            chunks_list.append({"original_index": index, "chunk": chunk})

In [23]:
chunks_df = pd.DataFrame(chunks_list)

In [24]:
print(chunks_df)

      original_index                                              chunk
0               1742  Reinforcement  Learning with  Function \nAppro...
1               1742  Although  there  are  convergent  online  algo...
2               1742  processes [2];  Q-Iearning with linear functio...
3               1742  gies,  this  paper  proves  that  the  iterate...
4               1742  lIn  a  ''trajectory-based''  algorithm,  the ...
...              ...                                                ...
6069            1892  [5] S. Edelman. Representation and recognition...
6070            1892  [9] D. J . Heeger, E. P. Simoncelli, and J. An...
6071            1892  2000. preprint. \n\n[13] S. Edelman and N. Int...
6072            1892  469, 1998. \n\n[17] A. Treisman. Perceiving an...
6073            1892  [21] E. Salinas and L. F. Abbott. Invariant vi...

[6074 rows x 2 columns]


In [25]:
import chromadb
chroma_client = chromadb.Client()

In [33]:
from langchain.embeddings import OpenAIEmbeddings
import os

# Initialize the OpenAI embeddings instance
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",  # Correct OpenAI's embedding model name
    openai_api_key=os.getenv("OPENAI_API_KEY")  # Your OpenAI API Key
)

chunks_df['embedding'] = chunks_df['chunk'].apply(lambda x: embeddings.embed_documents([x])[0])

client = chromadb.Client()
collection = client.get_or_create_collection(name="my_chunks")

# Add chunks and embeddings to the collection
# Generate unique IDs for each chunk (use the DataFrame index or generate custom IDs)
ids = [f"chunk-{i}" for i in chunks_df.index]

# Add chunks and embeddings to the collection
collection.add(
    ids=ids,  # Unique identifiers for each document
    documents=chunks_df['chunk'].tolist(),  # List of text chunks
    embeddings=chunks_df['embedding'].tolist(),  # List of embeddings
    metadatas=[{'original_index': idx} for idx in chunks_df.index]  # Metadata
)


IndexError: list index out of range